Using the artists schema, this script iterates over artistID's and gets the corresponding list of songID's, then appends each resulting (artistID,songID) pair to the song_ids.txt file.

In [1]:
# functions _get and get_artist_songs() in this code 
# are courtesy of GitHub user imdkm:
# https://gist.github.com/imdkm/a60247b59ff1881fa4bb8846a9b44c96

In [1]:
import requests, json
from time import sleep
import sys
import pandas as pd
import os

In [3]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [24]:
# constant values.
BASE_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = token
QUERY_SIZE = 100

In [25]:
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URL, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

def get_artist_songs(artist_id):
    # initialize variables & a list.
    current_page = 1
    next_page = True
    songs = []

    # main loop
    while next_page:

        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page}
        data = _get(path=path, params=params)

        page_songs = data['response']['songs']

        if page_songs:
            # add all the songs of current page,
            # and increment current_page value for next loop.
            songs += page_songs
            current_page += 1
        else:
            # if page_songs is empty, quit.
            next_page = False

    # get all the song ids, excluding not-primary-artist songs.
    songs = [song["id"] for song in songs
             if song["primary_artist"]["id"] == artist_id]

    return songs

In [26]:
def get_from(artist_id):
    with open("artists.txt","r") as f:
        lines = f.readlines()
        i=0
        while i < len(lines):
            if lines[i].split(',')[0] == str(artist_id):
                break
            i+=1
    return lines[i+1:]

In [36]:
# read in list of artist names
    
import os.path
if os.path.isfile('song_ids.txt'):
    l = !wc -l song_ids.txt
    if int(l[0].split()[0]) > 0:
        s = !tail -1 song_ids.txt
        last_artist = s[0].split(',')[0]
        artists = get_from(last_artist)
    
artist_ids = [line.strip().split(',')[0] for line in artists if line.strip().split(',')[0] != 'None']
print(str(len(artists))+" artists total")

2472 artists total


In [37]:
artist_set = set()
with open("song_ids.txt", "r") as f:
    for line in f:
        line = line.strip()
        artist_id,_ = line.split(',')
        artist_set.add(artist_id)

print("done reading")

done reading


In [38]:
len(artist_set)

569

In [ ]:
# query only a subset of artists at a time to not overwork server
# increment chunk_num on each run
    
artists_chunk = artist_ids[0:QUERY_SIZE]

# populate song ids using artist names
for i, artist_id in enumerate(artists_chunk):
    #sys.stdout.write('\r'+str(i).zfill(5))
    print(artist_id)
    
    # check if we've already queried this artist:
    if artist_id in artist_set:
        print(artist_id + " already queried, skipping")
        continue
    else:
        artist_set.add(artist_id)

    # get all song ids and make a list.
    song_ids = [None]
    try:
        song_ids = get_artist_songs(int(artist_id))
        print("-> " + str(len(song_ids))+" results")
    except:
        print("NOT FOUND")

    with open("song_ids.txt", "a") as f:
        for song_id in song_ids:
            f.write(str(artist_id)+","+str(song_id)+"\n")
            
    sleep(.5)
    
os.system('say "Done"')

13185
-> 30 results
483989
-> 2 results
354937
-> 30 results
453579
-> 19 results
14989
-> 8 results
1740819
-> 5 results
342527
-> 47 results
352574
-> 31 results
183884
-> 1 results
1421
1421 already queried, skipping
53342
-> 45 results
2219
-> 482 results
353591
-> 50 results
22822
-> 93 results
391995
-> 6 results
186655
-> 1 results
216609
-> 200 results
13531
-> 241 results
41921
41921 already queried, skipping
370172
-> 16 results
17675
-> 133 results
178
-> 220 results
1316142
-> 1 results
638714
-> 5 results
171547
-> 48 results
34171
-> 49 results
459759
-> 2 results
1307
-> 100 results
77
-> 320 results
335047
-> 7 results
457050
-> 1 results
214673
-> 36 results
279871
-> 88 results
654024
-> 35 results
18329
-> 78 results
351960
-> 14 results
44393
-> 164 results
67441
-> 41 results
21569
-> 76 results
45
45 already queried, skipping
378156
-> 5 results
554452
-> 17 results
1047981
-> 2 results
452358
-> 11 results
15130
-> 10 results
214663
-> 8 results
16377
-> 102 resu

In [ ]:
df = 